<a href="https://colab.research.google.com/github/castrostephano/CoinGeckoScrape/blob/main/MSI_Selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import glob
import os
import pandas as pd
from xlsxwriter import Workbook
import chromedriver_binary

AccountString_List = [{'Account_Number':'00','Account_Name': 'Budget_22'}]


for i in AccountString_List:

    PATH = (r"PathToYour\chromedriver.exe")
    driver = webdriver.Chrome(PATH)
    url = "http://msi.co.somerset.nj.us/SystemLogin.htm"
    driver.get(url)

    ## Login Screen
    #User ID and password provided by MSI
    user = driver.find_element(By.ID, "User")
    user.send_keys("userID#")
    user.send_keys(Keys.RETURN)
    time.sleep(0)
    password = driver.find_element(By.ID, "Password")
    password.send_keys("password")
    time.sleep(0)
    button = driver.find_element(By.ID, "Submit1").click()
    time.sleep(1)

    ## Home Screen
    # find the frame using id
    frame = driver.find_element(By.XPATH,"//frame[@id='_MAIN']")
    time.sleep(1)
    # switch the webdriver object to the iframe.
    driver.switch_to.frame(frame)
    #Click on report
    report = driver.find_element(By.XPATH,"//*[@id='menutable']/tbody/tr/td[7]").click()
    time.sleep(1)


    #Click on Account Activity Report - Choose Report Type
    AccountActivity = driver.find_element(By.XPATH,"//b[contains(.,'Budget Balance Report')]").click()
    time.sleep(1)
    #Click on Account String Box
    AccountString = driver.find_element(By.XPATH,"//input[@id='AccountStart']")
    #Type in Account String
    AccountString.send_keys(i['Account_Number'])
    time.sleep(2)
    #Check off print revenue accounts (it's checked my default)
    CheckRevAccount= driver.find_element(By.XPATH, "//input[@id='WithRev']").click()
    time.sleep(2)
    #Check off print with sub-accounts (it's checked my default)
    CheckSubAccount= driver.find_element(By.XPATH, "//input[@name='WithSub']").click()
    time.sleep(2)


    #Run report
    ActivityReport= driver.find_element(By.XPATH, "//input[@name='GetReport']").click()
    time.sleep(2)
    DownloadReport = driver.find_element(By.XPATH, "//a[contains(text(),'Click here to save/view CSV (Excel) Version')]").click()
    time.sleep(2)

    #Grab csv file from download folder
    List_of_Files = glob.glob(r'H:\Downloads\*.csv')
    latest_file = max(List_of_Files,default=None, key=os.path.getctime)

    #Turn csv into dataframe using pandas, skipping the extra rows at the top and bottom of report
    df = pd.read_csv(latest_file, on_bad_lines="skip", engine = 'python',encoding= 'unicode_escape',skiprows=6)

    # axis is 0 because we are referring to rows- would be 1 if columns, labels =1 is the index value
    # and inplace false because we want a result returned, not just to alter the db
    df = df.drop(axis=0, labels=1, inplace=False)

    # Use excel writer to convert the csv into an excel worksheet and save it in a folder
    with pd.ExcelWriter(f'FolderPath\{i["Account_Name"]}.xlsx', engine='xlsxwriter') as writer:
        df.to_excel(writer)

driver.quit
